# project ai: Easer

by Michiel Téblick and thibaut Van Goethem

In this notebook we will look at the easer model proposed at https://dl.acm.org/doi/pdf/10.1145/3308558.3313710.

This model will be applied to a dataset from foods.com which containes a bunch of recipes with user ratings/reactions on them.




In [1]:
import math

import pandas as pd
import numpy as np
import scipy
from sklearn.model_selection import KFold
import time
import pickle
from scipy import sparse
import statistics as st

## Reading and preprocessing the data

In [2]:
use_less_data = False


df_train = pd.read_csv('../folds/fold_0/train.csv')
df_test = pd.read_csv('../folds/fold_0/test.csv')
df_validate = pd.read_csv('../folds/fold_0/validate.csv')
df = pd.concat([df_train, df_test, df_validate])

print("amount of interactions in the full dataset: ",len(df))
print("amount of recipes in the full dataset: ",len(df.recipe_id.unique()))
print("amount of users in the full dataset: ",len(df.user_id.unique()))

if use_less_data:
    df = df[df['count_user'] >= 20]
    print("amount of recipes in the smaller dataset: ",len(df.recipe_id.unique()))
    print("amount of users in the smaller dataset: ",len(df.user_id.unique()))
df.reset_index()


amount of interactions in the full dataset:  733951
amount of recipes in the full dataset:  80511
amount of users in the full dataset:  32635


,index,user_id,recipe_id,date,rating,review,count_user,count_item
0,0,56680,79222,2006-11-11,5,"Oh, This was wonderful! Had a soup and salad ...",174,18
1,1,827374,79222,2010-11-29,3,We made this last night and really enjoyed it....,10,18
2,2,462571,208980,2007-07-05,5,These were a snap to whip up and were fantasti...,87,8
3,3,222139,208980,2007-09-08,5,I chose this recipe from Fall Pick A Chef. Co...,446,8
4,4,423539,342209,2009-06-02,5,These are lovely cookies not bland at all. \r\...,35,8
...,...,...,...,...,...,...,...,...
733946,151908,96177,196735,2009-01-12,5,We just loved these tatters. Quick easy and ve...,563,17
733947,151909,573325,196735,2010-09-08,5,"What a great, healthy, easy and yummy recipe!<...",880,17
733948,151910,203111,213546,2007-06-28,5,Very good potatoes! I served them with fried ...,217,6
733949,151911,41468,82303,2006-09-01,5,WOW this was great. What I love the most is th...,30,13


Set all ratings to 1 (even negative interactions are seen as interactions)

In [3]:
df.loc[:,'rating'] = 1

Below here are two ways to cut down on the amount of interactions that are used in this notebook
- The first one randomly removes x% of the users,
- The second one removes all user and recipes that have less than X amount of interaction containing them

We opted for the second form as this is more representative of how the models should be used due to the lower amount if recipes but more reactions per recipe. Also the second choice is a deterministic way of removing data, which the first one is not.
This does end up mostly giving slightly worse result compared to the first choice.

The reason we need to remove data is because a matrix inversion is done, which can not be done in a smart way.
Also the result of the inversion is not necessarily a sparse matrix so the full calculation needs to be done on dense matrices. This end up scaling O(n^3) in time complexity and O(n^2) for memory needed. n here is the amount of recipes.
So running on the full dataset would require more than 200gb of ram which we do not have.


### rescaling the id's
The recipes and users don't go from 0 to amount so if we were to put this in a matrix we would get empty columns and rows. This is not that handy so we reindex both the user_id and recipe_ids

This is a step we must not forget when entering the data in the model, as we also need to remap our input data using the same remapping that was used here

In [4]:
userSet = set(df['user_id'].to_list())
user_transform_dict = dict(map(reversed, enumerate(userSet)))
recipeSet = set(df['recipe_id'].to_list())
recipe_transform_dict = dict(map(reversed, enumerate(recipeSet)))
recipe_dict = dict(enumerate(recipeSet))

In [5]:
keep_nan_user = [k for k, v in user_transform_dict.items() if pd.isnull(v)]
keep_nan_recipe = [k for k, v in recipe_transform_dict.items() if pd.isnull(v)]


def transform_id(dataframe):
    tochange = dataframe['user_id']
    dataframe['user_id'] = tochange.map(user_transform_dict).fillna(tochange.mask(tochange.isin(keep_nan_user)))

    tochange = dataframe['recipe_id']
    dataframe['recipe_id'] = tochange.map(recipe_transform_dict).fillna(tochange.mask(tochange.isin(keep_nan_recipe)))
    return dataframe

def open_csv(filename, use_less_data=False):
    df = pd.read_csv(filename)
    if use_less_data:
        df = df[df['count_user'] >= 20]
    df = transform_id(df)
    df.loc[:,'rating'] = 1
    df.drop('review', axis=1, inplace=True)
    return df


### Creation of the folds


In [6]:
k = 10
folds = list()
for directory in ["../folds/fold_%d" % i for i in range(k)]:
    folds.append(( directory + "/train.csv", directory + "/validate.csv",directory + "/test.csv"))

## Creation model
Here we define the models used for the experiments. Both the easer predictor and a populaliry predictor are created. the popularity predictor is used as a baseline

In [7]:
def split_test(data_set):
    ground_truth = data_set.sort_values('date').groupby('user_id').tail(1)
    predict = pd.concat([data_set, ground_truth]).drop_duplicates(keep=False)
    return predict, ground_truth

def data_frame_to_matrix(dataframe):
    ratings = dataframe.rating
    idx = (dataframe.recipe_id, dataframe.user_id)
    return sparse.csc_matrix((ratings, idx), shape=(len(df.recipe_id.unique()), len(df.user_id.unique())),
                                dtype=float)

In [8]:
class popularity:
    def __init__(self):
        pass
    def train(self, data):
        data = data.sort_values('count_user',ascending=False)
        self.pop = data[data.columns[1]].to_numpy()
    def predict(self):
        return self.pop

In [9]:
class Easer:
    def __init__(self):
        pass

    def train(self, X_train, lambda_=1250):
        #Code here is a modified version of the code provided in the paper

        G = X_train.T.dot(X_train)
        G = G.toarray()
        diagIndices = np.diag_indices(G.shape[0])
        G[diagIndices] += lambda_
        P = scipy.linalg.inv(G)
        del G
        div = -np.diag(P)
        self.B = P / div
        self.B[diagIndices] = 0

    def predict(self, xu):
        return xu * self.B


In [10]:
K = 5
K2 = 10
K3 = 20

def recal_easer(model, predict_data, test_data):
    total = len(test_data)

    X_train = data_frame_to_matrix(predict_data)
    y_pred = model.predict(X_train)
    y_pred = y_pred.T

    X_test = data_frame_to_matrix(test_data)
    X_test = X_test.T

    interacted_recipes = (X_train.T == 1).toarray()
    y_pred[interacted_recipes] = -100000
    idx_top_scores = (-y_pred).argsort()[:,:20]
    dense_X_test = X_test.toarray()

    correct_K = 0
    correct_K2 = 0
    correct_K3 = 0
    ndcg_K = 0
    ndcg_K2 = 0
    ndcg_K3 = 0

    for idx, row in enumerate(idx_top_scores):
        for rank, index in enumerate(row):
            if dense_X_test[idx][index] == 1:
                if rank < K:
                    correct_K += 1
                    ndcg_K += 1/(math.log2(rank+2))
                if rank < K2:
                    correct_K2 += 1
                    ndcg_K2 += 1/(math.log2(rank+2))
                if rank < K3:
                    correct_K3 += 1
                    ndcg_K3 += 1/(math.log2(rank+2))


    print("easer recall@5,10,20 = %s,%s,%s" % (str(correct_K / total),str(correct_K2 / total),str(correct_K3 / total)))
    print("easer ndcg@5,10,20 = %s,%s,%s" % (str(ndcg_K / total),str(ndcg_K2 / total),str(ndcg_K3 / total)), end="\n\n")

    return correct_K/total, correct_K2/total,correct_K3/total, ndcg_K/total,ndcg_K2/total,ndcg_K3/total

## training models + evaluation


In [11]:
#Please enter the path here of where you will place the pickle files (with trailing /)
data_path="../results_aiproject/"
result_list_pop_K5=list()
result_list_pop_K10=list()
result_list_pop_K20=list()
result_list_pop_ndcg5=list()
result_list_pop_ndcg10=list()
result_list_pop_ndcg20=list()

for f_idx, fold_files in enumerate(folds):
    start = time.time()
    train_data = open_csv(fold_files[0], use_less_data)
    test_data = open_csv(fold_files[2], use_less_data)
    #Here we have the user item matrix
    interactions, ground_truth = split_test(test_data)
    interactions= pd.concat([interactions, train_data])
    X_train = data_frame_to_matrix(interactions)

    #train models

    model = Easer()
    model.train(X_train, lambda_=1250)

    recall5,recall10,recall20, ndcg5,ndcg10,ndcg20 = recal_easer(model, interactions, ground_truth)

    result_list_pop_K5.append(recall5)
    result_list_pop_K10.append(recall10)
    result_list_pop_K20.append(recall20)
    result_list_pop_ndcg5.append(ndcg5)
    result_list_pop_ndcg10.append(ndcg10)
    result_list_pop_ndcg20.append(ndcg20)

    print("done fold:",str(f_idx))

    end = time.time()
    print("training took : ", end - start, "s")
    # break

print("mean recall@5,10,20 over 10 folds: %s,%s,%s" % (str(st.mean(result_list_pop_K5)),str(st.mean(result_list_pop_K10)),str(st.mean(result_list_pop_K20))))
print("mean ndcg@5,10,20 over 10 folds: %s,%s,%s" % (str(st.mean(result_list_pop_ndcg5)),str(st.mean(result_list_pop_ndcg10)),str(st.mean(result_list_pop_ndcg20))), end="\n\n")
print("standard deviation recall@5,10,20 over 10 folds: %s,%s,%s" %(str(st.pstdev(result_list_pop_K5)), str(st.pstdev(result_list_pop_K10)), str(st.pstdev(result_list_pop_K20))))
print("standard deviation ndcg@5,10,20 over 10 folds: %s,%s,%s" % (str(st.pstdev(result_list_pop_ndcg5)),str(st.pstdev(result_list_pop_ndcg10)),str(st.pstdev(result_list_pop_ndcg20))))

easer recall@5,10,20 = 0.015931372549019607,0.023590686274509803,0.032475490196078434
easer ndcg@5,10,20 = 0.012638756900694333,0.015091048027232615,0.01733567293121932

done fold: 0
training took :  1641.1826887130737 s
easer recall@5,10,20 = 0.011948529411764705,0.017769607843137254,0.02971813725490196
easer ndcg@5,10,20 = 0.008929985435472609,0.01076754476122627,0.01380640650794218

done fold: 1
training took :  1503.3938691616058 s
easer recall@5,10,20 = 0.01715686274509804,0.02542892156862745,0.03553921568627451
easer ndcg@5,10,20 = 0.011732188296572622,0.014422104368790007,0.01693861185821921

done fold: 2
training took :  1482.9207170009613 s
easer recall@5,10,20 = 0.01685049019607843,0.023590686274509803,0.03308823529411765
easer ndcg@5,10,20 = 0.013006481531668007,0.015164477837351947,0.017534437185416998

done fold: 3
training took :  1649.4020981788635 s
easer recall@5,10,20 = 0.015318627450980392,0.021752450980392156,0.030637254901960783
easer ndcg@5,10,20 = 0.0110622659891